In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm
import time
import re
import sys
from selenium.webdriver.common.keys import Keys
keys = Keys()

In [3]:
url = 'https://search.shopping.naver.com/search/all?query=%EB%AC%B4%EC%84%A0%EC%9D%B4%EC%96%B4%ED%8F%B0&cat_id=&frm=NVSHATC' # 네이버 쇼핑 카테고리 url 입력
driver = webdriver.Chrome("C:/Users/Feldrance/anaconda3/chromedriver.exe")
driver.get(url)
time.sleep(2)
body = driver.find_element_by_tag_name('body')
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/ul/li[2]/a').click() ## 가격비교 분류
time.sleep(2)
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/div/div[1]/a[5]').click() ## 리뷰 많은 순 정렬
time.sleep(1)

for i in range (0,7): # 7회 스크롤, 스크롤 하지 않을 경우 에러 발생
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [4]:
p_btn = [] # 카테고리 페이지 내 제품
rpage_btn = [] # 리뷰 페이지

for i in range(1,41): # 카테고리 내에서 크롤링 할 제품 개수 (현재 40개)
    p_btn.append('div[' + str(i) + ']') 

for i in range(2,12): # 제품 당 크롤링 할 리뷰 페이지 수(페이지 당 리뷰 20개)
    rpage_btn.append('a:nth-child(' + str(i) + ')')

for i in p_btn:
    a = "//*[@id=\"__next\"]/div/div[2]/div/div[3]/div[1]/ul/div/" + i + "/li/div[1]/div[2]/div[1]/a"
    driver.find_element_by_xpath(a).click() ## 제품 페이지 열기
    time.sleep(2)

    rating_list = [] # 제품 바뀔때마다 리스트 초기화
    review_list = [] # 제품 바뀔때마다 리스트 초기화
    label_list = [] # 제품 바뀔때마다 리스트 초기화
    driver.switch_to.window(driver.window_handles[1]) # 크롤링 할 새 창으로 창 전환
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    pname = soup.find_all('h2')[2].text # 제품명 크롤링 (엑셀 저장시 사용)
    pname = pname.replace("/", "") # 제품명에 '/' 존재시 에러 발생하므로 제거
    for i in range(0, 5): # 리뷰 페이지 10개(10, 20, 30...) 크롤링 마다 다음(11~, 21~, 31~로 넘김, 현재 50페이지까지 크롤링)
        for pagenum in rpage_btn:
            driver.find_element_by_css_selector('#section_review > div.pagination_pagination__2M9a4 > '+pagenum).send_keys(keys.ENTER) # 리뷰 페이지 넘김
            time.sleep(2)
            for i in range(0,20):  # 제품 페이지 당 리뷰 20개
                review = driver.find_element_by_xpath('//*[@id="section_review"]/ul/li[' + str(i+1) + ']/div[2]/div[1]/p').text
                review = re.sub('[^#0-9a-zA-Zr-ㅣ가-힣 ]',"",review) # 불필요한 문자 제거
                review_list.append(review)
                rating = int(driver.find_element_by_xpath('//*[@id="section_review"]/ul/li[' + str(i+1) + ']/div[1]').text[2]) # 평점 크롤링
                rating_list.append(rating)
                if rating >= 3:
                    label = 1
                else:
                    label = 0
                label_list.append(label)
        driver.find_element_by_css_selector('#section_review > div.pagination_pagination__2M9a4 > a.pagination_next__3ycRH').send_keys(keys.ENTER) # 페이지 10개 크롤링 후 다음으로 넘김
        
driver.close() # 크롤링 종료 시 창 종료
driver.switch_to.window(driver.window_handles[0]) # 기본 카테고리 창으로 전환
res = pd.DataFrame({'ratings': rating_list, 'reviews': review_list, 'label': label_list})
res.to_excel(pname + ' review_crawling.xlsx') # 엑셀로 출력